<a href="https://colab.research.google.com/github/angelv-salazar/escenarios-posnaturales/blob/main/preparaci%C3%B3n_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparación de Datasets (conjuntos de datos)

In [ ]:
#@title Importar Librerías
#@markdown Librerías requeridas para la manipulación de imágenes
!apt install imagemagick > /dev/null 2>&1

import glob
import os
from math import floor, ceil
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
from fastai.vision import verify_images



In [ ]:
#@title Definir Funciones

#@markdown Funciones para la manipulación de imágenes

def crop_center(img, ratio):
  img = img.copy()
  width, height = img.size
  crop_size= ceil(ratio * min(img.size))
 
  left = (width - crop_size)/2
  top = (height - crop_size)/2
  right = (width + crop_size)/2
  bottom = (height + crop_size)/2

  return img.crop((left, top, right, bottom))

def save_crop(img, cnt, path, augment=False):
    img = img.copy()

    img.save(f'{path}{cnt:08d}.png')
    cnt += 1
    if augment:
        img.transpose(Image.FLIP_LEFT_RIGHT).save(f'{path}/{cnt:08d}.png')
        cnt += 1
        img.transpose(Image.FLIP_TOP_BOTTOM).save(f'{path}/{cnt:08d}.png')
        cnt += 1
        img.transpose(Image.ROTATE_180).save(f'{path}/{cnt:08d}.png')
        cnt += 1

    return cnt

def thumbnail(img, size=256):
    """
    resize image so smallest side will be equal to size
    """
    
    img = img.copy()

    if img.mode not in ('L', 'RGB'):
        img = img.convert('RGB')

    width, height = img.size

    if width == height:
        img.thumbnail((size, size), Image.ANTIALIAS)

    elif height > width:
        ratio = float(height) / float(width)
        newheight = ratio * size
        img = img.resize((size, int(floor(newheight))), Image.ANTIALIAS)

    elif width > height:
        ratio = float(width) / float(height)
        newwidth = ratio * size
        img = img.resize((int(floor(newwidth)), size), Image.ANTIALIAS)


    return img

def get_crop_bboxes(w, h):
    """
    calculate bounding boxes based on width and height
    """
    n_step = ceil(float(w) / float(h))
    shift = (w - h) // n_step
    shifting_array = []
    for step in range(0, n_step):
        if w > h:
            shifting = (shift * step, 0, shift * step + h, h)
        else:
            shifting = (0, shift * step, w, shift * step + w)
        shifting_array.append(shifting)
    if w > h:
        shifting = (w - h, 0, w, h)
    else:
        shifting = (0, h - w, w, h)
    shifting_array.append(shifting)
    return shifting_array

In [ ]:
#@title Montar Google Drive
#@markdown Monte Google Drive para cargar su Dataset, editar las imágenes y guardar los resultados.

#@markdown Después de ejecutar esta celda, obtendrá el enlace. Haga click en el enlace, otorgue acceso a su Drive y copie el código de autenticación.

#@markdown Pegue el código en la entrada a continuación y presione Enter

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Cortar centro
#@markdown Esta celda recortará el centro de cada imagen en el directorio de entrada, lo redimensionará y guardará el resultado en el directorio de salida

#@markdown Directorio de entrada.
input_dir = '' #@param {type:"string"}
#@markdown Directorio de salida. Si no existe será creado.
out_dir = '' #@param {type:'string'}
#@markdown Cambiar el tamaño a:
resize = "1024" #@param ["256", "512", "1024"] {allow-input: true}
resize = int(resize)
#@markdown Proporción del tamaño de corte. El tamaño del cuadrado recortado será igual a: "proporción  * tamaño del  lado mas corto".  `ratio * min side size`
ratio = 1 #@param {type:"slider", min:0.1, max:1, step:0.05}
#@markdown Aumentar el Dataset con rotación. *Mantenerlo **DESACTIVADO** para este taller*.
augment = False #@param {type:"boolean"}

if out_dir[-1] != "/":
  out_dir += "/"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

images = glob.glob(f'{input_dir}/*.tif')+glob.glob(f'{input_dir}/*.png')+glob.glob(f'{input_dir}/*.jpg')+glob.glob(f'{input_dir}/*.bmp')
cnt=0
print(f'Cropping {len(images)} images\n')
for image in tqdm(images):
  img = Image.open(image)
  cropped = crop_center(img, ratio)
  resized = cropped.resize((resize, resize), Image.ANTIALIAS)
  cnt=save_crop(resized, cnt, out_dir, augment)
print(f'Number of images saved: {cnt}')

In [ ]:
#@title Fraccionar Imágenes

#@markdown Esta celda cambiará el tamaño de cada imagen en el directorio de entrada, la dividirá en varias imágenes cuadradas y guardará el resultado en el directorio de salida

#@markdown Directorio de entrada
input_dir = '' #@param {type:'string'}
#@markdown Directorio de salida. Si no existe será creado.
out_dir = '' #@param {type:'string'}
#@markdown Cambiar el tamaño a:
resize = "512" #@param [256, 512, 1024] {allow-input: true}
resize = int(resize)
#@markdown Aumentar el Dataset con rotación. *Mantenerlo **DESACTIVADO** para este taller*.
Aumentar = False #@param {type:"boolean"}

if out_dir[-1] != "/":
  out_dir += "/"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

images = glob.glob(f'{input_dir}/*.tif')+glob.glob(f'{input_dir}/*.png')+glob.glob(f'{input_dir}/*.jpg')+glob.glob(f'{input_dir}/*.bmp')
cnt=0
print(f'Cropping {len(images)} images\n')
for image in tqdm(images):
    img = Image.open(image)
    img = thumbnail(img, size=resize)
    w, h = img.size
    bboxes = get_crop_bboxes(w, h)
    for bbox in bboxes:
        img_cropped = img.crop(bbox)
        cnt = save_crop(img_cropped, cnt, out_dir, False)
print(f'Number of images saved: {cnt}')

In [ ]:
#@title Verificar Imágenes
#@markdown Esta celda verificará las imágenes en el directorio de entrada. Las imágenes se redimensionarán al tamaño definido, se convertirán a RGB y se guardarán en el directorio de salida

#@markdown Directorio de entrada.
input_dir = '' #@param {type:'string'}
#@markdown Directorio de salida. Si no existe será creado.
out_dir = '' #@param {type:'string'}
#@markdown Cambiar el tamaño a:
resize = "1024" #@param [256, 512, 1024] {allow-input: true}
resize = int(resize)

if out_dir[-1] != "/":
  out_dir += "/"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

images = glob.glob(f'{input_dir}/*.tif')+glob.glob(f'{input_dir}/*.png')+glob.glob(f'{input_dir}/*.jpg')+glob.glob(f'{input_dir}/*.bmp')
cnt=0
print(f'Verifying {len(images)} images\n')
for image in tqdm(images):
    img = Image.open(image)
    img = img.resize((resize, resize), Image.ANTIALIAS)
    imgarr = np.array(img)
    img_channels = 1 if len(imgarr.shape) == 2 else imgarr.shape[2]
    if img_channels == 1:
      # print(imgarr.shape)
      R = np.stack((imgarr, imgarr, imgarr), axis=2)
      img = Image.fromarray(R, 'RGB')
      imgarr = np.array(img)
      # print(imgarr.shape)
    if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
      bg = Image.new('RGB', img.size, (255, 255, 255))
      
      bg.paste(img, (0, 0), img)
      img = bg
    img.save(f'{out_dir}{cnt:08d}.png')
    cnt+=1
    

In [ ]:
#@title Mejora automática de imágenes



#@markdown Directorio de entrada
input_dir = '' #@param {type:'string'}
#@markdown Directorio de salida. Si no existe será creado.
out_dir = '' #@param {type:'string'}
#@markdown Opciones de mejora:
auto_level = True #@param {type:"boolean"}
auto_gamma = False #@param {type:"boolean"}
invert = False #@param {type:"boolean"}

options = ''
options+=' -auto-level' if auto_level else ''
options+=' -auto-gamma' if auto_gamma else ''
options+=' -negate' if invert else ''

if out_dir[-1] != "/":
  out_dir += "/"

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

images = glob.glob(f'{input_dir}/*.tif')+glob.glob(f'{input_dir}/*.png')+glob.glob(f'{input_dir}/*.jpg')+glob.glob(f'{input_dir}/*.bmp')
print(f'Enhancing {len(images)} images\n')
for image in tqdm(images):
  outfile = f"{out_dir}{image.split('/')[-1]}"
  !convert {image} {options} {outfile}